In [ ]:
#mount the google drive to colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install category_encoders
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# unzip the file to the current directory
!unzip "/content/drive/MyDrive/data.zip"

Archive:  /content/drive/MyDrive/data.zip
  inflating: test_month_1.csv        
  inflating: test_month_2.csv        
  inflating: test_month_3.csv        
  inflating: train_month_1.csv       
  inflating: train_month_2.csv       
  inflating: train_month_3_with_target.csv  


In [ ]:
# import panda to open the csv files
import pandas as pd
train_month_3=pd.read_csv("train_month_3_with_target.csv")
test_month_3=pd.read_csv("test_month_3.csv")
train_month_1=pd.read_csv("train_month_1.csv")
train_month_2=pd.read_csv("train_month_2.csv")
test_month_1=pd.read_csv("test_month_1.csv")
test_month_2=pd.read_csv("test_month_2.csv")

In [ ]:
new_features_one_month_change=train_month_1.iloc[:,1:29]
new_features_one_month_change_test=test_month_1.iloc[:,1:29]
new_features_two_month_change=train_month_2.iloc[:,1:29]
new_features_two_month_change_test=test_month_2.iloc[:,1:29]

In [ ]:
categories=['homebanking_active',
 'has_homebanking',
 'has_insurance_21',
 'has_insurance_23',
 'has_life_insurance_fixed_cap',
 'has_life_insurance_decreasing_cap',
 'has_fire_car_other_insurance',
 'has_personal_loan',
 'has_mortgage_loan',
 'has_current_account',
 'has_pension_saving',
 'has_savings_account',
 'has_current_account_starter',
 'has_savings_account_starter']

In [ ]:
for i in ["homebanking_active","has_homebanking","has_insurance_21","has_insurance_23","has_life_insurance_fixed_cap","has_life_insurance_decreasing_cap","has_fire_car_other_insurance","has_personal_loan","has_mortgage_loan","has_current_account","has_pension_saving","has_savings_account","has_current_account_starter","has_savings_account_starter"]:
       categories.append("One_month_change_in_{}".format(i))
       categories.append("Two_month_change_in_{}".format(i)) 

In [ ]:
column_names=new_features_one_month_change.columns
for i in column_names:
     if i!="client_id":
          train_month_3["One_month_change_in_{}".format(i)]=train_month_3[i]-new_features_one_month_change[i]
          train_month_3["Two_month_change_in_{}".format(i)]=train_month_3[i]-new_features_two_month_change[i]
          test_month_3["One_month_change_in_{}".format(i)]=test_month_3[i]-new_features_one_month_change_test[i]
          test_month_3["Two_month_change_in_{}".format(i)]=test_month_3[i]-new_features_two_month_change_test[i]

# data preprocessing 



In [ ]:
column_names=new_features_one_month_change.columns
for i in column_names:
     if i!="client_id":
          train_month_3["One_month_change_in_{}".format(i)]=train_month_3[i]-new_features_one_month_change[i]
          train_month_3["Two_month_change_in_{}".format(i)]=train_month_3[i]-new_features_two_month_change[i]
          test_month_3["One_month_change_in_{}".format(i)]=test_month_3[i]-new_features_one_month_change_test[i]
          test_month_3["Two_month_change_in_{}".format(i)]=test_month_3[i]-new_features_two_month_change_test[i]

In [ ]:
# crate new variablae of age as of 2018TT
train_month_3=train_month_3.assign(age_at_tt=2018-pd.to_datetime(train_month_3.customer_birth_date).dt.year)
test_month_3=test_month_3.assign(age_at_tt=2018-pd.to_datetime(test_month_3.customer_birth_date).dt.year)
# create  new two variables of years of the relationship on test set and train set
train_month_3=train_month_3.assign(years_of_the_relationship=2018-pd.to_datetime(train_month_3.customer_since_all).dt.year) 
train_month_3=train_month_3.assign(years_of_the_relationship_bank_product=2018-pd.to_datetime(train_month_3.customer_since_bank).dt.year)
test_month_3=test_month_3.assign(years_of_the_relationship=2018-pd.to_datetime(test_month_3.customer_since_all).dt.year) 
test_month_3=test_month_3.assign(years_of_the_relationship_bank_product=2018-pd.to_datetime(test_month_3.customer_since_bank).dt.year)
# drop the varaible customer_education 
#train_month_3=train_month_3.drop(["customer_education"],axis=1)
#test_month_3=test_month_3.drop(["customer_education"],axis=1)

In [ ]:
# train and test split on train_three_month
from sklearn.model_selection import train_test_split
train, test = train_test_split(train_month_3, test_size=0.2)

In [ ]:
from operator import xor
def missing_indctor(x):
        if pd.isna(x):
           return 1
        else:
           return 0

In [ ]:
train["missing_customer_education"]=train["customer_education"].apply(lambda x:missing_indctor(x))
test_month_3["missing_customer_education"]=test_month_3["customer_education"].apply(lambda x: missing_indctor(x))
test["missing_customer_education"]=test["customer_education"].apply(lambda x: missing_indctor(x))

train["customer_education"].fillna(train["customer_education"].mode()[0], inplace=True)
test_month_3["customer_education"].fillna(train["customer_education"].mode()[0], inplace=True)
test["customer_education"].fillna(train["customer_education"].mode()[0], inplace=True)

In [ ]:
# replace the varaible customer_education 
train["missing_customer_occupation_code"]=train["customer_occupation_code"].apply(lambda x:missing_indctor(x))
test_month_3["missing_customer_occupation_code"]=test_month_3["customer_occupation_code"].apply(lambda x: missing_indctor(x))
test["missing_customer_occupation_code"]=test["customer_occupation_code"].apply(lambda x: missing_indctor(x))

train["customer_occupation_code"].fillna(train["customer_occupation_code"].mode()[0], inplace=True)
test_month_3["customer_occupation_code"].fillna(train["customer_occupation_code"].mode()[0], inplace=True)
test["customer_occupation_code"].fillna(train["customer_occupation_code"].mode()[0], inplace=True)

In [ ]:
train["customer_relationship_missing_ness"]=train["customer_relationship"].apply(lambda x:missing_indctor(x))
test_month_3["customer_relationship_missing_ness"]=test_month_3["customer_relationship"].apply(lambda x: missing_indctor(x))
test["customer_relationship_missing_ness"]=test["customer_relationship"].apply(lambda x: missing_indctor(x))

train["customer_relationship"].fillna(train["customer_relationship"].mode()[0], inplace=True)
test_month_3["customer_relationship"].fillna(train["customer_relationship"].mode()[0], inplace=True)
test["customer_relationship"].fillna(train["customer_relationship"].mode()[0], inplace=True)

In [ ]:
train["customer_children_missing_ness"]=train["customer_children"].apply(lambda x:missing_indctor(x))
test_month_3["customer_children_missing_ness"]=test_month_3["customer_children"].apply(lambda x: missing_indctor(x))
test["customer_children_missing_ness"]=test["customer_children"].apply(lambda x: missing_indctor(x))

train["customer_children"].fillna(train["customer_children"].mode()[0], inplace=True)
test_month_3["customer_children"].fillna(train["customer_children"].mode()[0], inplace=True)
test["customer_children"].fillna(train["customer_children"].mode()[0], inplace=True)

In [ ]:
train["years_of_the_relationship_missing_ness"]=train["years_of_the_relationship"].apply(lambda x:missing_indctor(x))
test_month_3["years_of_the_relationship_missing_ness"]=test_month_3["years_of_the_relationship"].apply(lambda x: missing_indctor(x))
test["years_of_the_relationship_missing_ness"]=test["years_of_the_relationship"].apply(lambda x: missing_indctor(x))


train["years_of_the_relationship"].fillna(train["years_of_the_relationship"].mean(), inplace=True)
test_month_3["years_of_the_relationship"].fillna(train["years_of_the_relationship"].mean(), inplace=True)
test["years_of_the_relationship"].fillna(train["years_of_the_relationship"].mean(), inplace=True)


In [ ]:
train["years_of_the_relationship_bank_product_missing_ness"]=train["years_of_the_relationship_bank_product"].apply(lambda x:missing_indctor(x))
test_month_3["years_of_the_relationship_bank_product_missing_ness"]=test_month_3["years_of_the_relationship_bank_product"].apply(lambda x: missing_indctor(x))
test["years_of_the_relationship_bank_product_missing_ness"]=test["years_of_the_relationship_bank_product"].apply(lambda x: missing_indctor(x))



train["years_of_the_relationship_bank_product"].fillna(train["years_of_the_relationship_bank_product"].mean(), inplace=True)
test_month_3["years_of_the_relationship_bank_product"].fillna(train["years_of_the_relationship_bank_product"].mean(), inplace=True)
test["years_of_the_relationship_bank_product"].fillna(train["years_of_the_relationship_bank_product"].mean(), inplace=True)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# drop variables
test_month_3=test_month_3.drop(["customer_birth_date","customer_since_all","customer_since_bank"],axis=1)
test=test.drop(["customer_birth_date","customer_since_all","customer_since_bank"],axis=1)
train=train.drop(["customer_birth_date","customer_since_all","customer_since_bank"],axis=1)

In [ ]:
encoder = ce.WOEEncoder()
X=train.iloc[:,train.columns!="target"]
X=X.iloc[:,X.columns!="client_id"]

In [ ]:
y=test.iloc[:,test.columns!="target"]
y=y.iloc[:,y.columns!="client_id"]

In [ ]:
encoder.fit(X, train["target"])
X_cleaned = encoder.transform(y)

In [ ]:
X_cleaned

,homebanking_active,has_homebanking,has_insurance_21,has_insurance_23,has_life_insurance_fixed_cap,has_life_insurance_decreasing_cap,has_fire_car_other_insurance,has_personal_loan,has_mortgage_loan,has_current_account,...,Two_month_change_in_visits_distinct_so_areas,age_at_tt,years_of_the_relationship,years_of_the_relationship_bank_product,missing_customer_education,missing_customer_occupation_code,customer_relationship_missing_ness,customer_children_missing_ness,years_of_the_relationship_missing_ness,years_of_the_relationship_bank_product_missing_ness
30708,0,0,0,0,0,0,1,0,0,0,...,0.0,64,5.0,1.0,1,0,0,0,0,0
19415,0,0,0,0,0,0,0,0,0,1,...,0.0,100,23.0,23.0,1,0,1,1,0,0
36632,0,0,0,0,0,0,0,0,0,0,...,0.0,50,29.0,29.0,1,0,0,1,0,0
26455,0,0,0,0,0,0,0,0,0,0,...,0.0,54,14.0,14.0,1,0,1,1,0,0
21789,0,0,0,0,0,0,0,0,0,1,...,0.0,93,13.0,1.0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37692,0,0,0,0,0,0,0,0,0,1,...,0.0,73,37.0,37.0,1,0,0,1,0,0
20046,1,1,0,0,0,1,1,0,1,1,...,0.0,54,35.0,35.0,0,0,0,0,0,0
20824,0,1,1,0,0,0,1,0,0,1,...,0.0,52,39.0,1.0,1,0,0,0,0,0
28769,0,1,1,0,0,1,1,0,0,0,...,0.0,36,12.0,11.0,1,0,0,0,0,0


# using percentile coding for the ordinal variables 

In [ ]:
train.describe()
# seperate the features and the target
train_X=train.loc[:, train.columns!="target"]
train_X=train_X.loc[:, train_X.columns!="client_id"]
train_Y=train.loc[:, "target"]
test_X=test.loc[:, test.columns!="target"]
test_Y=test.loc[:, "target"]

In [ ]:
#using the oversampling strategy for the class imbalance
!sudo pip install imbalanced-learn

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler 
oversample = RandomOverSampler(sampling_strategy="minority")
X_res, y_res = oversample.fit_resample(train_X, train_Y)

In [ ]:
# hyperprameter tunning
parameters = {
    "splitter":["best","random"],
    "max_features":["auto", "sqrt", "log2"],
    'max_depth': range(50,100,1),
    'min_samples_leaf':range(1,100,1),
    'criterion': ["gini", "entropy"],
    #"random_state":range(1,100,1)
}

In [ ]:
 from sklearn.metrics import confusion_matrix
 from sklearn.metrics import fbeta_score, make_scorer

In [ ]:
clf =DecisionTreeClassifier(random_state = 42)
tuning_model=GridSearchCV(clf,param_grid=parameters,cv=10,verbose=3,scoring="accuracy")
tuning_model= tuning_model.fit(X_res, y_res)

Streaming output truncated to the last 5000 lines.
[CV 4/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.849 total time=   0.2s
[CV 5/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.864 total time=   0.3s
[CV 6/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.858 total time=   0.3s
[CV 7/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.863 total time=   0.3s
[CV 8/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.854 total time=   0.3s
[CV 9/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.849 total time=   0.3s
[CV 10/10] END criterion=gini, max_depth=50, max_features=auto, min_samples_leaf=26, splitter=best;, score=0.847 total time=   0.3s
[CV 1/10] END criterion=gini, m

KeyboardInterrupt: ignored

In [ ]:
#now build the descision_tree model and test the model
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
kk=0
D={}
for i in parameters["splitter"]:
   for k in parameters["max_features"]:
       for l in parameters["max_depth"]:
            for z in parameters["min_samples_leaf"]:
               for p in parameters['criterion']:
                 #for hh in parameters["random_state"]:
                      clf =DecisionTreeClassifier(splitter=i,max_features=k,min_samples_leaf=z,criterion=p,max_depth=l,random_state=10)
                      clf.fit(X_res, y_res)
                      predicted=clf.predict(test_X.iloc[:,test_X.columns!="client_id"])
                      fpr, tpr, thresholds = metrics.roc_curve(test_Y, predicted, pos_label=None)
                      a=metrics.auc(fpr, tpr)
                      if a>kk:
                            kk=a
                            D["best"]=[i,k,l,z,p]
                            print(a)
#tuning_model=GridSearchCV(clf,param_grid=parameters,cv=10,verbose=3)
#tuning_model= tuning_model.fit(X_res, y_res)

NameError: ignored

In [ ]:
D

{'best': ['best', 'auto', 6, 75, 'gini']}

In [ ]:
tuning_model.best_params_

{'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 5}

In [ ]:
#now test the data
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#clf =AdaBoostClassifier(base_estimator=LogisticRegression,n_estimators=200,algorithm='SAMME')

clf = DecisionTreeClassifier(criterion="gini",max_depth=6,min_samples_leaf=75,max_features="auto",random_state=42,splitter="best")
clf.fit(X_res, y_res)
predicted=clf.predict(test_X.iloc[:,test_X.columns!="client_id"])

In [ ]:
# evaluation using ROC CURVE
from sklearn.metrics import confusion_matrix
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_Y, predicted, pos_label=None)
metrics.auc(fpr, tpr)
confusion_matrix(test_Y, predicted)[1,1]/(confusion_matrix(test_Y, predicted)[1,0]+confusion_matrix(test_Y, predicted)[1,1])

0.6564885496183206

In [ ]:
tpr

array([0.        , 0.65648855, 1.        ])

In [ ]:
metrics.auc(fpr, tpr)

0.658364951896712

In [ ]:
a=confusion_matrix(test_Y, predicted)

In [ ]:
a[1,0]

156

In [ ]:
test_predicted=clf.predict_proba(test_month_3.iloc[:,test_month_3.columns!="client_id"])[:,1]

In [ ]:
test_predicted

array([0.        , 0.10822306, 0.84915773, ..., 0.23765902, 0.46124292,
       0.23765902])

In [ ]:
ID=test_month_3["client_id"]

In [ ]:
ID

0        ccf4cd93d5c32cd8a59809d54b4d53ac
1        56605a660d18549592653ff6941186f1
2        bda5f84c05e5695a7ec10550b457890f
3        a2f1c04bc3acf2222e658a897400798f
4        e83aadc3b0d25dbc12a35551afa25807
                       ...               
27295    1a634b5cfe6dbf35e77ce6519c0e3939
27296    84b7af2ddbeb919ab238ef7e6a83936c
27297    c257c09e3f4445d67b52a1007b0205df
27298    fdd7a1b7e58f642a1fc4867b8f7cdf3a
27299    9b088b828fa94419c00175b6536857f9
Name: client_id, Length: 27300, dtype: object

In [ ]:
test_predicted

array([0.        , 0.10822306, 0.84915773, ..., 0.23765902, 0.46124292,
       0.23765902])

In [ ]:
test_predicted=np.stack((ID, test_predicted), axis=1)

In [ ]:
test_predicted=pd.DataFrame(test_predicted)

In [ ]:
test_predicted.to_csv("/content/drive/MyDrive/test.csv",index=False)

In [ ]:
import numpy as np 
X_train=train_simple_drop_all.loc[:,train_simple_drop_all.columns!="target"]
Y_train=train_simple_drop_all.loc[:, "target"]

In [ ]:
X_test=test_simple_drop_all.loc[:,test_simple_drop_all.columns!="target"]

In [ ]:
!git clone -b dev --single-branch https://github.com/kearnz/autoimpute.git
!cd autoimpute
!python setup.py install

Cloning into 'autoimpute'...
remote: Enumerating objects: 2462, done.
remote: Total 2462 (delta 0), reused 0 (delta 0), pack-reused 2462
Receiving objects: 100% (2462/2462), 1.63 MiB | 6.33 MiB/s, done.
Resolving deltas: 100% (1730/1730), done.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X_train,Y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train_fs, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score
yhat = model.predict(X_test_fs)

accuracy = accuracy_score(test_simple_drop_all.loc[:,"target"], yhat)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 94.25


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_simple_drop_all.loc[:,"target"],yhat)
metrics.auc(fpr, tpr)

0.5041666666666667

In [ ]:
test_month_3.columns

In [ ]:
test_month_3=fs.transform(test_month_3)

Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
clf = RandomForestClassifier(random_state=0)

#X=train_simple_drop_all.loc[:, train_simple_drop_all.columns!="target"]
#X=X.drop(["customer_since_all","customer_since_bank","customer_birth_date","has_homebanking","homebanking_active"],axis=1)
#convert the cartigorical variables to dummy variaables 
#X["customer_children"] = ord_enc.fit_transform(X[["customer_children"]])
#X["customer_relationship"] = ord_enc.fit_transform(X[["customer_relationship"]])
#Y=train_simple_drop_all.loc[:,  "target"]
clf = clf.fit(X_train_fs, Y_train)
#X=test_simple_drop_all.loc[:, test_simple_drop_all.columns!="target"]
#X=X.drop(["customer_since_all","customer_since_bank","customer_birth_date","has_homebanking","homebanking_active"],axis=1)
#convert the cartigorical variables to dummy variaables 
#X["customer_children"] = ord_enc.fit_transform(X[["customer_children"]])
#X["customer_relationship"] = ord_enc.fit_transform(X[["customer_relationship"]])
Y=test_simple_drop_all.loc[:,  "target"]
predicted=clf.predict(X_test_fs)
fpr, tpr, thresholds = metrics.roc_curve(Y, predicted, pos_label=None)
metrics.auc(fpr, tpr)


0.5

In [ ]:
predicted=clf.predict(test_month_3)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: ignored

In [ ]:
sum(predicted==1)

120

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
X["customer_children"] = ord_enc.fit_transform(X[["customer_children"]])

In [ ]:
for i in X.columns:
          print(i)

homebanking_active
has_homebanking
has_insurance_21
has_insurance_23
has_life_insurance_fixed_cap
has_life_insurance_decreasing_cap
has_fire_car_other_insurance
has_personal_loan
has_mortgage_loan
has_current_account
has_pension_saving
has_savings_account
has_savings_account_starter
has_current_account_starter
bal_insurance_21
bal_insurance_23
cap_life_insurance_fixed_cap
cap_life_insurance_decreasing_cap
prem_fire_car_other_insurance
bal_personal_loan
bal_mortgage_loan
bal_current_account
bal_pension_saving
bal_savings_account
bal_savings_account_starter
bal_current_account_starter
visits_distinct_so
visits_distinct_so_areas
customer_gender
customer_occupation_code
customer_self_employed
customer_education
customer_children
customer_relationship
age_at_tt
post_code_count
years_of_the_relationship
years_of_the_relationship_bank_product
